In [1]:
import numpy as np
import scipy.linalg as sla

In [2]:
dataset = {
    'A': np.array([
        [ 1., 2., 0., 1., 0.,  4., -1., -3.],
        [ 1., 3., 0., 0., 1., -1., -1.,  2.],
        [ 1., 4., 1., 0., 0.,  2., -2.,  0.],
    ]),
    'b': np.array([4., 5., 6.]),
    'B': np.array([
        [ 1., 1., -1., 0.,  3.,  4., -2.,  1.],
        [ 2., 6.,  0., 0.,  1., -5.,  0., -1.],
        [-1., 2.,  0., 0., -1.,  1.,  1.,  1.],
    ]),
    'd': np.array([7., 3., 3.]),
    'D': np.array([
        [ 6.,  11., -1., 0.,  6.,  -7., -3., -2.],
        [11.,  41., -1., 0.,  7., -24.,  0., -3.],
        [-1.,  -1.,  1., 0., -3.,  -4.,  2., -1.],
        [ 0.,   0.,  0., 0.,  0.,   0.,  0.,  0.],
        [ 6.,   7., -3., 0., 11.,   6., -7.,  1.],
        [-7., -24., -4., 0.,  6.,  42., -7., 10.],
        [-3.,   0.,  2., 0., -7.,  -7.,  5.,  1.],
        [-2.,  -3., -1., 0.,  1.,  10., -1.,  3.],
    ]),
    'c': np.array([-10., -31., 7., 0., -21., -16., 11., -7.]),
    'J': [2, 3, 4],
}
init_plan = np.array([0., 0., 6., 4., 5., 0., 0., 0.])
eps=1e-9

In [82]:
def solve(dataset, init_plan, verbose=True, verbose_iter=2, max_iter=1000):
    A = dataset['A'].copy()
    b = dataset['b'].copy()
    B = dataset['B'].copy()
    d = dataset['d'].copy()
    D = dataset['D'].copy()
    c = dataset['c'].copy()
    J_base = [i for i in dataset['J']]
    J_ast = [i for i in dataset['J']]
    Jn = [i for i in range(A.shape[1]) if i not in J_base]
    x = init_plan.copy()
    
    if verbose:
        print('\n\n Initial data \n\n')
        print(f'Initial plan: {x}')
        print(f'A = {A}')
        print(f'b = {b}')
        print(f'B = {B}')
        print(f'J_base: {J_base}')
        print(f'J*: {J_ast}')
    
    skip_to_step3 = False
    
    for it in range(max_iter):
        
        if it >= verbose_iter:
            verbose = False
        
        if verbose:
            print('\n===========================')
            print(f'\nIteration {it + 1}\n')
        if not skip_to_step3:

            # Step 1
            c_ = D @ x + c
            A_base = A[:, J_base]
            c_base = c_[J_base]
            u = -c_base @ sla.inv(A_base)
            approx = u.T @ A + c_

            if verbose:
                print(f'c_: {c_}')
                print(f'A base: {A_base}')
                print(f'c base: {c_base}')
                print(f'u: {u}')
                print(f'Delta: {approx}')

            # Step 2
            if np.all(approx[Jn] > -eps):
                print(f'Optimal plan found: {x}')
                print(f'cs = {c @ x + 0.5 * x @ D @ x}')
                return x

            j0 = np.argmin(approx)

            if verbose:
                print(f'j0: {j0}')
        
        skip_to_step3 = False
        
        # Step 3
        l = np.zeros(c.shape)
        l[j0] = 1.
        idxes = []
        D_ast = np.zeros((len(J_ast), len(J_ast)))
        A_ast = np.zeros((A.shape[0], len(J_ast)))
        for i in range(len(J_ast)):
            for j in range(len(J_ast)):
                D_ast[i, j] = D[J_ast[i], J_ast[j]]
        
        for j in range(len(J_ast)):
            A_ast[:, j] = A[:, J_ast[j]]
        
        H = np.concatenate((np.concatenate((D_ast, np.transpose(A_ast)), axis=1), 
                            np.concatenate((A_ast, np.zeros(A_base.shape)), axis=1)), axis=0)
        bb = np.concatenate((D[J_ast, j0], A[:, j0]), axis=0)
        print(bb)
        buffer = -sla.inv(H) @ bb
        l[J_ast] = buffer[:len(J_ast)]
        
        if verbose:
            print(f'l: {l}')
        
        # Step 4
        delta = l @ D @ l
        theta = []
        for i in J_ast:
            theta.append(np.inf if l[i] > -eps else -x[i] / l[i])
        
        theta.append(np.inf if delta < eps else np.abs(approx[j0]) / delta)
        theta = np.array(theta)
        j_ast = np.argmin(theta)
        if j_ast < theta.shape[0] - 1:
            j_ast = J_ast[j_ast]
        else:
            j_ast = j0
        theta0 = np.min(theta)
        
        if verbose:
            print(f'theta: {theta}')
            print(f'theta0: {theta0}')
            print(f'j*: {j_ast}')
        
        # Step 5
        x_new = x + theta0 * l
        if verbose:
            print(f'x_new: {x_new}')
        
        # Step 6
        if j_ast == j0:
            J_ast.append(j0)
        elif j_ast in J_ast and j_ast not in J_base:
            approx[j0] += delta
            J_ast.remove(j_ast)
            skip_to_step3 = True
        else:
            J_ = [i for i in J_ast if i not in J_base]
            s = J_base.index(j_ast)
            flag = True
            e_s = np.zeros(len(J_base))
            e_s[s] = 1.
            for j_ in J_:
                if np.abs(e_s @ A_base @ A[:, j_]) > eps:
                    flag = False
                    break
            
            J_ast.remove(j_ast)
            J_base.remove(j_ast)
            if not flag:
                J_base.append(j_)
                approx[j0] += delta * theta0
                skip_to_step3 = True
            elif sorted(J_ast) == sorted(J_base):
                J_ast.append(j0)
            else:
                J_base.append(j_)
                J_ast.append(j0)
        
        x = x_new
        
        if verbose:
            print(f'J_base: {J_base}')
            print(f'J_ast: {J_ast}')
        


In [83]:
_ = solve(dataset, init_plan)



 Initial data 


Initial plan: [0. 0. 6. 4. 5. 0. 0. 0.]
A = [[ 1.  2.  0.  1.  0.  4. -1. -3.]
 [ 1.  3.  0.  0.  1. -1. -1.  2.]
 [ 1.  4.  1.  0.  0.  2. -2.  0.]]
b = [4. 5. 6.]
B = [[ 1.  1. -1.  0.  3.  4. -2.  1.]
 [ 2.  6.  0.  0.  1. -5.  0. -1.]
 [-1.  2.  0.  0. -1.  1.  1.  1.]]
J_base: [2, 3, 4]
J*: [2, 3, 4]


Iteration 1

c_: [ 14.  -2.  -2.   0.  16. -10. -12.  -8.]
A base: [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
c base: [-2.  0. 16.]
u: [  0. -16.   2.]
Delta: [  0. -42.   0.   0.   0.  10.   0. -40.]
j0: 1
[-1.  0.  7.  2.  3.  4.]
l: [ 0.  1. -4. -2. -3.  0.  0.  0.]
theta: [1.5        2.         1.66666667 0.84      ]
theta0: 0.84
j*: 1
x_new: [0.   0.84 2.64 2.32 2.48 0.   0.   0.  ]
J_base: [2, 3, 4]
J_ast: [2, 3, 4, 1]


Iteration 2

c_: [ 11.48  18.16   1.36   0.     4.24 -31.84  -1.08  -9.68]
A base: [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
c base: [1.36 0.   4.24]
u: [ 0.   -4.24 -1.36]
Delta: [ 5.88000000e+00 -3.55271368e-15  0.00000000e+00  0.00000000e+00
  0.000